In [308]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [337]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 1
EVENT_NAME = 'CUSUM_0.0004'
LABEL_NAME = "TB"

START_DATE = "20210101"
END_DATE = "20241231"

RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "interm/events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_FEATURES.pkl"
EVENT_FILE_PATH = EVENTS_DIR / f"{RESAMPLED_NAME}_{EVENT_NAME}.pkl"

In [338]:
DIRECTION_LABEL_DIR = BASE_DIR / "labels/direction_labels"
DIRECTION_LABEL_DIR.mkdir(parents=True, exist_ok=True)
DIRECTION_LABEL_FILE_PATH = DIRECTION_LABEL_DIR / f"{RESAMPLED_NAME}-{EVENT_NAME}-{LABEL_NAME}.pkl"

In [339]:
%%time
df = pd.read_pickle(PROCESSED_FILE_PATH)
t_events = pd.read_pickle(EVENT_FILE_PATH)

CPU times: user 1.7 ms, sys: 143 ms, total: 145 ms
Wall time: 212 ms


In [340]:
df.loc[t_events].head()

,open,high,low,close,volume,spread,close_pct_return,close_return,close_log_return,close_fd_return,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-04 13:46:00,102.9590,102.9630,102.9435,102.9460,2.831500e+11,0.002205,-0.000131,-0.0135,-0.000131,1.894669,...,4,1,-0.258819,-0.965926,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 13:50:00,102.9690,102.9910,102.9690,102.9910,2.191600e+11,0.002043,0.000209,0.0215,0.000209,1.937369,...,4,1,-0.258819,-0.965926,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 13:55:00,102.9910,103.0235,102.9910,103.0235,3.020600e+11,0.002191,0.000325,0.0335,0.000325,1.952365,...,4,1,-0.258819,-0.965926,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 14:00:00,103.0275,103.0370,103.0020,103.0100,3.462000e+11,0.002282,-0.000165,-0.0170,-0.000165,1.902438,...,4,1,-0.500000,-0.866025,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 14:16:00,103.0345,103.0555,103.0345,103.0555,5.678500e+11,0.001925,0.000218,0.0225,0.000218,1.947650,...,4,1,-0.500000,-0.866025,0.0,1.0,0.724793,0.688967,0.5,0.866025


## Calculate Target Volatility (trgt)
This will be used for the calculation of dynamic TP/SL


In [341]:
import pandas_ta as ta

In [342]:
# 預計持單 1 小時 (60 minutes)
# 計算 1 小時內價格可能到哪裡
# 計算 1 小時 normalized atr
atr12 = df.ta.natr(length=12)

In [343]:
atr12

timestamp
2021-01-04 13:33:00         NaN
2021-01-04 13:34:00         NaN
2021-01-04 13:35:00         NaN
2021-01-04 13:36:00         NaN
2021-01-04 13:37:00         NaN
                         ...   
2024-12-30 23:55:00    0.008614
2024-12-30 23:56:00    0.007976
2024-12-30 23:57:00    0.008346
2024-12-30 23:58:00    0.008075
2024-12-30 23:59:00    0.007959
Name: NATR_12, Length: 1486378, dtype: float64

In [344]:
trgt = atr12.reindex(t_events, method='ffill') / 100 # rate

In [345]:
(trgt).describe()

count    141392.000000
mean          0.000277
std           0.000216
min           0.000019
25%           0.000151
50%           0.000225
75%           0.000331
max           0.003497
Name: NATR_12, dtype: float64

## Apply Triple Barrier
We use daily volatility to calculate TP/SL, and we set our T1 as 1day

In [346]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import pandas as pd

def applyPtSlOnT1(close,events,ptSl,molecule):
    # apply stop loss/profit taking, if it takes place before t1 (end of event)
    events_=events.loc[molecule]
    out=events_[['t1']].copy(deep=True)
    
    pt = ptSl[0] * events_['trgt'] if ptSl[0] > 0 else pd.Series(index=events.index)
    sl = -ptSl[1] * events_['trgt'] if ptSl[1] > 0 else pd.Series(index=events.index)

        
    for loc, t1 in events_['t1'].fillna(close.index[-1]).items():
        df0=close[loc:t1] # path prices
        df0=(df0/close[loc]-1)*events_.at[loc,'side'] # path returns (rate)
        out.loc[loc,'sl']=df0[df0<sl[loc]].index.min() # earliest stop loss
        out.loc[loc,'pt']=df0[df0>pt[loc]].index.min() # earliest profit taking
        
    return out


def parallel_apply(func, items, num_threads=4, **kwargs):
    
    def worker(molecule):
        return func(molecule=molecule, **kwargs)

    chunks = np.array_split(items, num_threads)
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(executor.map(worker, chunks))

    return pd.concat(results).sort_index()


def getEvents(close, tEvents, ptSl, trgt, minTrgt, numThreads=4, t1=False, side=None):
    # Step 1: Filter targets
    trgt = trgt.loc[tEvents]
    trgt = trgt[trgt > minTrgt]

    # Step 2: Set vertical barrier (t1)
    if t1 is False:
        t1 = pd.Series(pd.NaT, index=tEvents)

    # Step 3: Build events DataFrame
    if side is None:
        side_, ptSl_ = pd.Series(1., index=trgt.index), [ptSl[0], ptSl[0]]
    else:
        side_, ptSl_ = side.loc[trgt.index], ptSl[:2]

    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': side_}, axis=1).dropna(subset=['trgt'])

    # Step 4: Apply barriers in parallel
    df0 = parallel_apply(
        func=applyPtSlOnT1,
        items=events.index,
        num_threads=numThreads,
        close=close,
        events=events,
        ptSl=ptSl_
    )

    # Step 5: Choose the first touched barrier
    events['t1'] = df0.dropna(how='all').min(axis=1)
    
    if side is None:
        events = events.drop('side', axis=1)
        
    return events


In [347]:
def get_vertical_barrier(t_events, close, delta=pd.Timedelta(minutes=60)):
    """
    For each event in t_events, find the timestamp in close.index 
    that is at least delta later. Returns a pd.Series of t1.
    """
    barrier_times = t_events + delta
    t1_idx = close.index.searchsorted(barrier_times)
    valid_idx = t1_idx[t1_idx < len(close)]
    t1 = pd.Series(close.index[valid_idx], index=t_events[:len(valid_idx)])
    return t1

In [348]:
t1 = get_vertical_barrier(t_events, df['close'], delta=pd.Timedelta(minutes=120))

In [349]:
t1

timestamp
2021-01-04 13:46:00   2021-01-04 15:46:00
2021-01-04 13:50:00   2021-01-04 15:50:00
2021-01-04 13:55:00   2021-01-04 15:55:00
2021-01-04 14:00:00   2021-01-04 16:00:00
2021-01-04 14:16:00   2021-01-04 16:16:00
                              ...        
2024-12-30 20:29:00   2024-12-30 22:29:00
2024-12-30 20:34:00   2024-12-30 22:34:00
2024-12-30 20:55:00   2024-12-30 22:56:00
2024-12-30 21:06:00   2024-12-30 23:06:00
2024-12-30 21:45:00   2024-12-30 23:45:00
Name: timestamp, Length: 141387, dtype: datetime64[ns]

In [350]:
trgt.describe()

count    141392.000000
mean          0.000277
std           0.000216
min           0.000019
25%           0.000151
50%           0.000225
75%           0.000331
max           0.003497
Name: NATR_12, dtype: float64

In [351]:
minTrgt = trgt.quantile(.3)
minTrgt
# minTrgt = 0

np.float64(0.00016535322367973722)

In [352]:
%%time

events = getEvents(
    close=df['close'],
    tEvents=t_events, 
    ptSl=[1,1], 
    trgt=trgt, 
    minTrgt=minTrgt, 
    numThreads=20, 
    t1=t1, 
    side=None
)

CPU times: user 27.7 s, sys: 2.64 s, total: 30.4 s
Wall time: 28.2 s


In [353]:
events.head()

,t1,trgt
timestamp,,
2021-01-04 14:00:00,2021-01-04 14:02:00,0.000183
2021-01-04 14:21:00,2021-01-04 14:24:00,0.000167
2021-01-04 14:35:00,2021-01-04 14:41:00,0.000176
2021-01-04 14:51:00,2021-01-04 15:05:00,0.000166
2021-01-04 15:07:00,2021-01-04 15:19:00,0.000197


In [354]:
events = events.dropna().copy()

# Keep only intraday events

In [355]:
events = events[events.index.date==events.t1.dt.date]

## Create classification labels

In [356]:
def getBins(events, close, minRet=0.0, t1=None):
    '''
    Compute event's outcome (including side information, if provided).
    events is a DataFrame where:
    -events.index is event's starttime
    -events['t1'] is event's endtime
    -events['trgt'] is event's target
    -events['side'] (optional) implies the algo's position side
    -t1 is original vertical barrier series
    Case 1: ('side' not in events): bin in (-1,1) <-label by price action
    Case 2: ('side' in events): bin in (0,1) <-label by pnl (meta-labeling)
    '''
    # 1) prices aligned with events
    events_ = events.dropna(subset=['t1'])
    px = events_.index.union(events_['t1'].values).drop_duplicates()
    px = close.reindex(px, method='bfill')
    # 2) create out object
    out = pd.DataFrame(index=events_.index)
    out['ret'] = px.loc[events_['t1'].values].values / px.loc[
        events_.index] - 1
    if 'side' in events_: out['ret'] *= events_['side']  # meta-labeling
    out['bin'] = np.sign(out['ret'])

    if 'side' not in events_:
        # only applies when not meta-labeling.
        # to update bin to 0 when vertical barrier is touched, we need the
        # original vertical barrier series since the events['t1'] is the time
        # of first touch of any barrier and not the vertical barrier
        # specifically. The index of the intersection of the vertical barrier
        # values and the events['t1'] values indicate which bin labels needs
        # to be turned to 0.
        vtouch_first_idx = events[events['t1'].isin(t1.values)].index
        out.loc[vtouch_first_idx, 'bin'] = 0.

    if 'side' in events_: out.loc[out['ret'] <= 0, 'bin'] = 0  # meta-labeling
    out.loc[out['ret'].abs() < minRet, 'bin'] = 0. # 不做獲利過少的交易 (設為 0)
    # out = out.loc[out['ret'].abs() >= minRet] # 過濾掉獲利過少的交易 (刪掉)
    return out

In [357]:
labels = getBins(events, df['close'], minRet=0, t1=t1)

In [358]:
print(labels.ret.abs().describe())
print(labels.ret.abs().quantile(.3))

count    98766.000000
mean         0.000501
std          0.000399
min          0.000010
25%          0.000293
50%          0.000395
75%          0.000570
max          0.013808
Name: ret, dtype: float64
0.0003115995193163634


In [359]:
labels.bin.value_counts()

bin
 1.0    41553
-1.0    40544
 0.0    16669
Name: count, dtype: int64

In [360]:
labels["bin_class"] = labels["bin"] + 1

In [361]:
labeled_events = events.join(labels, how='inner')

In [362]:
labeled_events.head()

,t1,trgt,ret,bin,bin_class
timestamp,,,,,
2021-01-04 14:00:00,2021-01-04 14:02:00,0.000183,0.000228,1.0,2.0
2021-01-04 14:21:00,2021-01-04 14:24:00,0.000167,-0.000214,-1.0,0.0
2021-01-04 14:35:00,2021-01-04 14:41:00,0.000176,-0.000199,-1.0,0.0
2021-01-04 14:51:00,2021-01-04 15:05:00,0.000166,-0.000223,-1.0,0.0
2021-01-04 15:07:00,2021-01-04 15:19:00,0.000197,0.000267,1.0,2.0


## Save events to disk

In [363]:
labeled_events.to_pickle(DIRECTION_LABEL_FILE_PATH)

In [364]:
DIRECTION_LABEL_FILE_PATH

PosixPath('../data/labels/direction_labels/USDJPY-1m-20210101-20241231-CUSUM_0.0004-TB.pkl')